# Imports

In [1]:
from NCA.NCA import *
from NCA.utils import *
import os

# Setup

In [2]:
DEVICE = "cuda:0"
HEIGHT = 50
WIDTH = 50
CHANNELS = 16
REPETITIONS = 100
PATH_TO_IMAGE = "../Images/lizard.png" #<--- image to grow 
IMAGE_NAME = "Lizard" #<--- name of image for saving purposes 

In [3]:
path = "Trained_models/Lizard"
masking_order = [0,1,2,3,4,5,6,7,8,9,10,11,12] #<--- should be the same as the masking order in experiments
damage_sites = [[15,7], [35,16], [15,42], [31,34]] #<--- change depending on experiment 
ncas = [DummyVCA,ReducedCA, MaskedCA]

In [4]:
reference_image,seed = get_reference_image_and_seed(PATH_TO_IMAGE, HEIGHT,WIDTH,CHANNELS)

# Path Creation for Saving Data

In [5]:
path_to_folder = "Regrow_exp_data/" + IMAGE_NAME 
if not os.path.exists(path_to_folder):
    os.makedirs(path_to_folder)
    print(f"Path: {path_to_folder} created")
else: 
    print(f"Path: {path_to_folder} already exists, all OK!")

Path: Regrow_exp_data/Lizard already exists, all OK!


In [6]:
for id, (root, subdirs, files) in enumerate(os.walk(path, topdown=True)):
    if len(files) != 0:
        loss_log = []
        files.sort(key=natural_keys)
        for idx, file in enumerate(files):
            loss_sites= []
            print(root)
            nca = ncas[id - 1](CHANNELS, hidden_n=96, mask_n=masking_order[idx])
            nca.load_state_dict(torch.load(root + "/" + file))
            nca.to(DEVICE).eval()
            for k in range(REPETITIONS):
                for dmg_site in damage_sites:
                    x = seed.clone()
                    for i in range(2048):
                        x = nca(x)
                        x = x.detach()
                        if (i != 0) and (i %1024 ==0):
                            mask = create_circular_mask([HEIGHT,WIDTH], dmg_site[1], dmg_site[0], 10)
                            x = x * ~mask[None, None, ...]
                    loss = (reference_image - x[:, :4, :, :]).pow(2).sum()
                    loss_sites.append(loss.log().item())
                print(f"Model {file}, Repetition: {k}/{REPETITIONS}, num_losses {len(loss_sites)}")
            loss_log.append(loss_sites)
        np_loss = np.asarray(loss_log)
        np.save(path_to_folder + type(nca).__name__ , np_loss )

Trained_models/Lizard/DummyVCA


/tmp/ipykernel_1810744/2277248245.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  nca.load_state_dict(torch.load(root + "/" + file))


Model DummyVCA_padding_0.pth, Repetition: 0/100, num_losses 4
Model DummyVCA_padding_0.pth, Repetition: 1/100, num_losses 8
Model DummyVCA_padding_0.pth, Repetition: 2/100, num_losses 12
Model DummyVCA_padding_0.pth, Repetition: 3/100, num_losses 16
Model DummyVCA_padding_0.pth, Repetition: 4/100, num_losses 20
Model DummyVCA_padding_0.pth, Repetition: 5/100, num_losses 24
Model DummyVCA_padding_0.pth, Repetition: 6/100, num_losses 28
Model DummyVCA_padding_0.pth, Repetition: 7/100, num_losses 32
Model DummyVCA_padding_0.pth, Repetition: 8/100, num_losses 36
Model DummyVCA_padding_0.pth, Repetition: 9/100, num_losses 40
Model DummyVCA_padding_0.pth, Repetition: 10/100, num_losses 44
Model DummyVCA_padding_0.pth, Repetition: 11/100, num_losses 48
Model DummyVCA_padding_0.pth, Repetition: 12/100, num_losses 52
Model DummyVCA_padding_0.pth, Repetition: 13/100, num_losses 56
Model DummyVCA_padding_0.pth, Repetition: 14/100, num_losses 60
Model DummyVCA_padding_0.pth, Repetition: 15/100, nu